This is following:  A Neural Algorithm of Artistic Style

Leon A. Gatys, Alexander S. Ecker, Matthias Bethge

--> https://arxiv.org/abs/1508.06576

Style transfer is an optimisation problem, used to  minimisation the loss function. Taking two images (style image and x image ), the optimisation minimises the difference betweeen the images.





Zero shot or one shot learning: machine learning with very little data. 

Network used is VGG16: it is an image classification convolutional network 
At each layer it apples a series of operations to the input image
Input image: matrix of values 

At each layer we have a stack of filters that are learned overtime
Filters: the filters are 3D vectors, which are a collection of matrics that are 2d, and the 3D is the rgb
At each of these filters, a matrix multiplication and then a summation operation occurs on te input image.
It acts as feture identifer
large number imples a feture is detected 
If not = zero.

It will output a feature map or an activation map (large matrix of values). We want to minimise the difference between our feature map and our content image.


Content image: Image to modify 
Style image: we want to apply to content image
Mixed Image: empty, random nosie initilase dimage we will add to over time.

There is style loss and content loss and the combination of these using gradient value updates the mixed image.


In [1]:
# import our dependencies 

%matplotlib inline
import matplotlib.pyplot as plt # plot out what we see
import tensorflow as tf  # machine learnign library 
import numpy as np   # help calculate gram matrix
import PIL.Image    # show image we want to show

In [2]:
tf.__version__

'2.5.0'

In [3]:
#16 layer convostuionary netwoerk. It is prerained with neccesssary filters
# more layers means better predictions but mroe computation time 
#import vgg16.py

In [4]:
from keras.applications.vgg16 import VGG16
model = VGG16(weights='imagenet')
print(model.summary())

553476096/553467096 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [5]:
#vgg16.maybe_download()

In [6]:
# define image helper fucntions 

def load_image(filename, max_size=None):
    image = PIL.Image.open(filename)

    if max_size is not None:
        # Calculate the appropriate rescale-factor for
        # ensuring a max height and width, while keeping
        # the proportion between them.
        factor = max_size / np.max(image.size)
    
        # Scale the image's height and width.
        size = np.array(image.size) * factor

        # The size is now floating-point because it was scaled.
        # But PIL requires the size to be integers.
        size = size.astype(int)

        # Resize the image.
        image = image.resize(size, PIL.Image.LANCZOS)

    # Convert to numpy floating-point array.
    return np.float32(image)

In [7]:
def save_image(image, filename):
    # Ensure the pixel-values are between 0 and 255.
    image = np.clip(image, 0.0, 255.0)
    
    # Convert to bytes.
    image = image.astype(np.uint8)
    
    # Write the image-file in jpeg-format.
    with open(filename, 'wb') as file:
        PIL.Image.fromarray(image).save(file, 'jpeg')

In [8]:
def plot_image_big(image):
    # Ensure the pixel-values are between 0 and 255.
    image = np.clip(image, 0.0, 255.0)

    # Convert pixels to bytes.
    image = image.astype(np.uint8)

    # Convert to a PIL-image and display it.
    display(PIL.Image.fromarray(image))


In [9]:
def plot_images(content_image, style_image, mixed_image):
    # Create figure with sub-plots.
    fig, axes = plt.subplots(1, 3, figsize=(10, 10))

    # Adjust vertical spacing.
    fig.subplots_adjust(hspace=0.1, wspace=0.1)

    # Use interpolation to smooth pixels?
    smooth = True
    
    # Interpolation type.
    if smooth:
        interpolation = 'sinc'
    else:
        interpolation = 'nearest'

    # Plot the content-image.
    # Note that the pixel-values are normalized to
    # the [0.0, 1.0] range by dividing with 255.
    ax = axes.flat[0]
    ax.imshow(content_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Content")

    # Plot the mixed-image.
    ax = axes.flat[1]
    ax.imshow(mixed_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Mixed")

    # Plot the style-image
    ax = axes.flat[2]
    ax.imshow(style_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Style")

    # Remove ticks from all the plots.
    for ax in axes.flat:
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

Loss Function 



In [10]:
# define mean squared error:  it is a function given to input tensors (a, b)
# outputs a scaler --> single value

# it is a hlerper fucntion inside our content loss and our style loss

def mean_squared_error(a, b):
    return tf.reduce_mean(tf.square(a - b))

#it is the average of the sqaure of the difference between the output feature map and our image
# one tensor will be our content image or style image. The other will be our feature map
                #. --> the result of the chosen layer 
# the difeeec is subtracted, the squared then we do a reduce mean
# square makes it a positive

# reduce Mean: find the average of all vlaues inside of the matrix 
#The smaller the means squared error, the closer you are to finding the line of best fit.


In [11]:
# Creates the loss-function for the content-image.

'''Parameters:
    session: An open TensorFlow session for running the model's graph.
    model: The model, e.g. an instance of the VGG16-class.
    content_image: Numpy float array with the content-image.
    layer_ids: List of integer id's for the layers to use in the model. --> will be ther higher level layers
'''
#you should expirment what looks good for different layers
def create_content_loss(session, model, content_image, layer_ids):

  # it is a python dictionary object that is genrateed with placeholders as keys
  # it genrates a set of place holder values 
  # key is the image, value is the content image.
  # for very time we want to feed anything into our compeiutuon graph -->as ditionary
   feed_dict = model.create_feed_dict(image=content_image)

    # play around with differnet layers 
   layers = model.get_layer_tensors(layer_ids)

  # calaute the output values given the layers --> feed into dictoanry
  # get outptu values for thaose layers for the conent image
   values = session.run(layers, feed_dict=feed_dict)

   with model.graph.as_default():
     # need empty as we will claaues a collection of lossses 
     layer_losses = []


      # For each layer and its corresponding values
     for value, layer in model(values, layers):

        # tf.cocnsstnt --> cannont change
        # this is the content image value
        value_const = tf.constant(value)


        #claaclte loss using mean sqaured error
        # inputs: value at a layer, constant value
        #
        loss = mean_squared_error(layer, value_const)

        # add to pre inititlised list 
        layer_losses.append(loss)


     total_loss = tf.reduce_mean(layer_losses)


   return total_loss


# get eman sqaure eroro of feature activtions of the given layer in th model 
# between the conent image and lmixed image --.~ when minised, it will make the mixed image that much more stylised 



In [12]:
# for style we do a gram matrix
# gram matrix : we are measuring the correlations between our feature vectors after flattening the filter image into vectors
# it is the matrix product between our initial matirix and then its transposed ( filpping it by 90 degrees )
# i.e multiplying a matrix by its transpose 
# this is used to minimise via the mean sqaured error
# it is 4D tensors: collection of pixel and rgb make it 3d -->a collection of those make it 4D
def gram_matrix(tensor):

  #4d tensor from convolutional layer
  shape = tensor.get_shape()

  # each stack in a the matrox is called a channel.
  num_channels = int(shape[3])


  # -1 : means whatever number makes the data fit
  # we are reshapping th edensor soe it is a 2d matrix 
  # flatten the contents of each o the feature chnnels --> so we can multiplt it 
  # akin to normalisation 
  matrix = tf.reshape(tensor, shape=[-1, num_channels])


    # the 2-dim matrix with itself. 
    # matrix multipled by its transpose
  gram = tf.matmul(tf.transpose(matrix), matrix)

  return gram



In [13]:
# style loss 
# compute given the gram matrix


def create_style_loss(session, model, style_image, layer_ids):
  # creates place holder value for style image
  feed_dict = model.create_feed_dict(image=style_image)

  # then get layers 
  layers = model.get_layer_tensors(layer_ids)


  with model.graph.as_default():

      # this is deffernece to the content loss
      # we take the gram matrix of a layer, instead of the regular
      # why ? ( from crator at a confernce ): it encodes 2nd order ststisics of the set of filters --> looking at it froma higehr layer
      # gram maxtrix , tolss away anything uncewesay to focus on the syle 
      # hence it works for style and not content 
    gram_layers = [gram_matrix(layer) for layer in layers]


    values = session.run(gram_layers, feed_dict=feed_dict)


    # Initialize an empty list of loss-functions.
    layer_losses = []


    # For each Gram-matrix at each layer and its corresponding values or their loss function.
    for value, gram_layer in model(values, gram_layers):

      value_const = tf.constant(value)


      # where calducalting the mean sqaured error of the ground layer and the value of that ground matrix layer when inputting the style image
      loss = mean_squared_error(gram_layer, value_const)


      # list of loss-functions --> losses at each layer
      layer_losses.append(loss)
    
    # take the average of all losess in matrix --> scalar value 
    total_loss = tf.reduce_mean(layer_losses)

  return total_loss





In [14]:
# one more loss 
# the denoising loss( totoal vartioan loss) --> when paper didnt do this image was OK
                                          # when they did it was --> result improved
# shifts images by one pixel on the x and y axis and calculates the diffenec btween the shifted image the original image 
# abosloute value so it is positive 

#--> helps supress noise in the image that was generated : its a blurriness, not that clear 


def create_denoise_loss(model):
    loss = tf.reduce_sum(tf.abs(model.input[:,1:,:,:] - model.input[:,:-1,:,:])) + \
           tf.reduce_sum(tf.abs(model.input[:,:,1:,:] - model.input[:,:,:-1,:]))

    return loss

# Style-transfer 

The main optimaisation algorithm for the Style-Transfer algorithm, tha uses graduent descent on the prevoiusly defined loss fucntion and it normalises the loss function. 

Each itertaion of the optimisation, the losss-values are adjusted so they indviually equal to one. This allowws for the lodd-weights to be se independtly for the chosen sytel and conent layers. 

These weights are also adpated during optimisation, ensuring the presevration of the desired ratio between the style, conentand denoising. 

In [26]:
# we can modify the weigths 
# weighing the sytle more than the content
# To:DO --> learn the best weights later with learnig algorthim 
'''
parameters: 
    content_image: Numpy 3-dim float-array with the content-image.
    style_image: Numpy 3-dim float-array with the style-image.
    content_layer_ids: List of integers identifying the content-layers.
    style_layer_ids: List of integers identifying the style-layers.
    weight_content: Weight for the content-loss-function.
    weight_style: Weight for the style-loss-function.
    weight_denoise: Weight for the denoising-loss-function.
    num_iterations: Number of optimization iterations to perform.
    step_size: Step-size for the gradient in each iteration.

'''
def style_transfer(content_image, style_image,
                   content_layer_ids, style_layer_ids,
                   weight_content=1.5, weight_style=10.0, #these weights can be changed and modified
                   weight_denoise=0.3,                      
                   num_iterations=120, step_size=10.0):
  

  #model = vgg16.VGG16()

  model = VGG16() # define the model --> 16 layer convoluntary net with 
  # a fully conetneted layer at the end: usually used for classfication 

  # Create a TensorFlow-session.
  #the session always encapuates the computation graph 
  session = tf.InteractiveSession(graph=model.graph)

  # Print the names of the content-layers.
  # helps to see whats happening 
  print("Content layers:")
  print(model.get_layer_names(content_layer_ids))
  print()


  # Create the loss-function for the content-layers and -image.
  # (session value, moel value, content image, layer_id is indexies) as we are trying to minimise the 
  # loss for the content
  #layer_id: is indexies -->claeute the diffenec between the raw actiavation 
  #   at the given layers in the conenet image. This can be thought of as a matrix
  #The output loss cotnent value is a scalar value

  #Note: the gram matrix for content loss is not used, as the gram matrix is a 
  #      second order statistic. Style is more abstract than content, content is somehting that can be 
  #      pointed out and say what is the coent in an image, so the raw acvtivation in content can be taken
  #      style is more nuramnce and emebebd and engrained in every layer.
  #      the gram matrix creates that extra layer of abstration 
  loss_content = create_content_loss(session=session,
                                       model=model,
                                       content_image=content_image,
                                       layer_ids=content_layer_ids)
  
   # Create the loss-function for the denoising of the mixed-image.
   # the extra step is the gram matrix calcuation 
   # taking the raw activations, it calautates the gram matrixes from them.
  loss_style = create_style_loss(session=session,
                                     model=model,
                                     style_image=style_image,
                                     layer_ids=style_layer_ids)
  # Create the loss-function for the denoising of the mixed-image.
  # so it is a little less blurry
  loss_denoise = create_denoise_loss(model)

  # adjust levels of loss functions, normalize them
  # multiply them with a variable
  # taking reciprocal values of loss values of content, style, denoising
  # small constant to avoid divide by 0
  # adjustment value normalizes loss so approximately 1
  # weights should be set relative to each other dont depend on layers
  # we are using

  # Create TensorFlow variables for adjusting the values of
  # the loss-functions. This is explained below.
  # create adjustement weight- adjusts how much each are weighted

  adj_content = tf.Variable(1e-10, name='adj_content')
  adj_style = tf.Variable(1e-10, name='adj_style')
  adj_denoise = tf.Variable(1e-10, name='adj_denoise')


  # Initialize the adjustment values for the loss-functions.
  session.run([adj_content.initializer,
                 adj_style.initializer,
                 adj_denoise.initializer])
  

  # Create TensorFlow operations for updating the adjustment values.
  # this actually adjusts them
  # These are basically just the reciprocal values of the
  # loss-functions, with a small value 1e-10 added to avoid the
  # possibility of division by zero.
  # think of it as a biasis vector 

  update_adj_content = adj_content.assign(1.0 / (loss_content + 1e-10))
  update_adj_style = adj_style.assign(1.0 / (loss_style + 1e-10))
  update_adj_denoise = adj_denoise.assign(1.0 / (loss_denoise + 1e-10))


  # now combine the losses 
  # This is the weighted loss-function that will be minimised
  # below in order to generate the mixed-image.
  # Because the loss-values is multiplied with their reciprocal
  # adjustment values, relative weights can now be used for the
  # loss-functions that are easier to select, as they are
  # independent of the exact choice of style- and content-layers.

  loss_combined = weight_content * adj_content * loss_content + \
                    weight_style * adj_style * loss_style + \
                    weight_denoise * adj_denoise * loss_denoise

  # Now we will caalcute the gradiants using 
  # TensorFlow to get the mathematical function for the
  # gradient is not just one, but using multiel gardainet values
  # using the models input value and the combined loss
  # it gets the gradient of the combined loss-function with regard to
  # the input image. (mixed)
  # gradient value will give us a direction , can be up or down
  # i.e update our image up or down to minimise the loss

  gradient = tf.gradients(loss_combined, model.input)

  # List of tensors that we will run in each optimization iteration.
  # this is a collection of scalar values 
  run_list = [gradient, update_adj_content, update_adj_style, \
                update_adj_denoise]
  
  # The mixed-image is initialized with random noise.
  # It is the same size as the content-image.
  # where we first init it
  # np.random.rand --> initlaise an empty matrix of random values
  mixed_image = np.random.rand(*content_image.shape) + 128

  for i in range(num_iterations):

    # Create a feed-dict with the mixed-image.
    feed_dict = model.create_feed_dict(image=mixed_image)

    # Using TensorFlow --> calculate the value of the
    # gradient, as well as updating the adjustment values.
    # gradeint tells us how ot update out rimeg
    # gradirent is a slope value, so given a sigmoid , the gradient is a sttrght line that either points up or down
    # wehn gradients are taken and multipled by the image, it will update the image in someway
    # every time the loss is minismed, the gradient is going to be "better"
    # by better --> the gradient is going to minimise the difference between the output image and the style image.
    #               everytime it will be minimum until it hits the local minimum
    #               so the output image is goign to be the most stylised that it can be

    grad, adj_content_val, adj_style_val, adj_denoise_val \
    = session.run(run_list, feed_dict=feed_dict)

    # Reduce the dimensionality of the gradient.
    # Remove single-dimensional entries from the shape of an array.
    #
    # gradient value is reduced so that it is the same size, so it can be multiped by the matrix 
    grad = np.squeeze(grad)

    # Scale the step-size according to the gradient-values.
    #
    # same a learning rate
    # Ratio of weights:updates
    # akin to learning rate
    # scaled too slowly: never going to converge
    # sacle too fast: overfit
    # so nee to update the gradeient and weigtj at each layer so it converages properly
    step_size_scaled = step_size / (np.std(grad) + 1e-8)

    # Update the image by following the gradient.
    # gradient descent preformed 
    # grad--> scaler 
    # step_size_scaled--> scale vlaue
    # gives output mixed image that will look bad at first but the more update the more styles it will be
    mixed_image -= grad * step_size_scaled

    # Ensure the image has valid pixel-values between 0 and 255.
    # Given an interval, values outside the interval are clipped 
    # to the interval edges.
    # clip all values that is not beteen 0- 255--> want it to be rgb
    mixed_image = np.clip(mixed_image, 0.0, 255.0)

    # Print a little progress-indicator.

    print(". ", end="")

    # Display status once every 10 iterations, and the last.
    # print iterations and plot
    if (i % 10 == 0) or (i == num_iterations - 1):
        print()
        print("Iteration:", i)

        # Print adjustment weights for loss-functions.
        msg = "Weight Adj. for Content: {0:.2e}, Style: {1:.2e}, Denoise: {2:.2e}"
        print(msg.format(adj_content_val, adj_style_val, adj_denoise_val))
          # Plot the content-, style- and mixed-images.
        plot_images(content_image=content_image,style_image=style_image,mixed_image=mixed_image)
                  
      
   
   print()
   print("Final image")
   plot_image

   plot_image_big(mixed_image)

   # Close the TensorFlow session to release its resources.
   session.close()

   # Return the mixed-image.

   return mixed_image







  







IndentationError: ignored

# Images

First load the content-image that has the overall contours that are wanted in the mixed-image

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
content_filename = '/content/gdrive/My Drive/StyleTrans/willy_wonka_old.jpg'
content_image = load_image(content_filename, max_size=None)

Then load the stle-image: this has the colours and textures for the mixed-image

In [ ]:
style_filename = '/content/gdrive/My Drive/StyleTrans/style7.jpg'
style_image = load_image(style_filename, max_size=300)

In [ ]:
# define list of integreed which identify the layers in the nural network that will be used for matching the conent-imgse.
# they are indices into the layers in the neural network.
# for the VGG16 model, the 5th layer(ndex4) seems to work well as the sole content-layer


content_layer_ids = [4]


In [ ]:
# define anotehr list of integres for the style layers
# the VGG16-model has 13 convolutional layers.
# this selects all those layers as the style-layers.
# this is somewhat slow to optimise.

style_layer_ids = list(range(13))

# can also select a sub-set of the layers, e.g. like this:
# style_layer_ids = [1, 2, 3, 4]

In [ ]:
%%time
img = style_transfer(content_image=content_image,
                     style_image=style_image,
                     content_layer_ids=content_layer_ids,
                     style_layer_ids=style_layer_ids,
                     weight_content=1.5,
                     weight_style=10.0,
                     weight_denoise=0.3,
                     num_iterations=60,
                     step_size=10.0)

To:Do learnign algothim to learn the best weights